In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Deploy Staging Model to Production
# MAGIC 
# MAGIC - Staging 단계의 모델을 Production 단계로 전환

# COMMAND ----------

import os
from mlflow.tracking import MlflowClient
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ----------------------
# 설정
# ----------------------
REALTIME_MODEL_NAME = os.getenv("REALTIME_MODEL_NAME", "team4-realtime-model")
TARGET_STAGE = "Production"  # Production으로 전환

client = MlflowClient()

print(f"모델명: {REALTIME_MODEL_NAME}")
print(f"타겟 스테이지: {TARGET_STAGE}")

# COMMAND ----------

# 1️⃣ Staging 단계의 최신 모델 버전 확인
try:
    all_versions = client.search_model_versions(f"name='{REALTIME_MODEL_NAME}'")
    if not all_versions:
        raise ValueError(f"모델 '{REALTIME_MODEL_NAME}'이 존재하지 않습니다.")
    
    print("등록된 모델 버전:")
    for version in all_versions:
        print(f"   버전 {version.version}: {version.current_stage}")
    
    # Staging 단계의 모델 찾기
    staging_versions = [v for v in all_versions if v.current_stage == "Staging"]
    
    if not staging_versions:
        raise ValueError("Staging 단계의 모델이 없습니다.")
    
    # 가장 높은 버전 선택
    latest_staging = max(staging_versions, key=lambda x: int(x.version))
    model_version = latest_staging.version
    
    print(f"Staging 단계의 최신 버전: {model_version}")
    
except Exception as e:
    print(f"모델 버전 확인 실패: {e}")
    raise

# COMMAND ----------

# 2️⃣ Staging -> Production 전환
try:
    print(f"모델 버전 {model_version}을 {TARGET_STAGE}으로 전환 중...")
    
    client.transition_model_version_stage(
        name=REALTIME_MODEL_NAME,
        version=model_version,
        stage=TARGET_STAGE,
        archive_existing_versions=True  # 기존 Production 버전을 Archived로 전환
    )
    
    print(f"성공! 모델 버전 {model_version}이 {TARGET_STAGE} 단계로 전환되었습니다.")
    
except Exception as e:
    print(f"모델 스테이지 전환 실패: {e}")
    raise

# COMMAND ----------

# 3️⃣ 최종 확인
try:
    updated_versions = client.search_model_versions(f"name='{REALTIME_MODEL_NAME}'")
    
    print("\n최종 모델 상태:")
    for version in updated_versions:
        print(f"   버전 {version.version}: {version.current_stage}")
    
    # Production 단계 모델 확인
    production_versions = [v for v in updated_versions if v.current_stage == "Production"]
    if production_versions:
        prod_version = production_versions[0]
        print(f"\n배포 완료: {REALTIME_MODEL_NAME} 버전 {prod_version.version}이 Production에 배포되었습니다.")
    else:
        print("\nProduction 단계 모델을 확인할 수 없습니다.")
        
except Exception as e:
    print(f"최종 상태 확인 실패: {e}")